In [1]:
from langchain import hub
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import UnstructuredPDFLoader
from langchain.embeddings import OpenAIEmbeddings
from langchain.schema import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma

In [2]:
from dotenv import load_dotenv
import os

load_dotenv()

API_KEY = os.getenv("OPENAI_API_KEY")

In [3]:
loader = UnstructuredPDFLoader("resume.pdf")

data = loader.load()

C:\Users\amira\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(data)

In [7]:
embedding = OpenAIEmbeddings(model='text-embedding-ada-002', api_key=API_KEY)

In [8]:
vectorstore = Chroma.from_documents(documents=splits, embedding=embedding)
retriever = vectorstore.as_retriever()

In [13]:
job_description = """
Below are the skills needed for this project. Students with the following relevant skills and interest, regardless of major, are encouraged to apply! This is a team based multidisciplinary project. Students on the team are not expected to have experience in all areas, but should be willing to learn and will be asked to perform a breadth of tasks throughout the two semester project.
Advanced Data Science and Modeling Techniques
Specific Skills: Applied project experience with Large Language Models and other applied AI techniques OR advanced coursework

Likely Majors: DATA, STATS, MATH, CS


Data Science
Specific Skills: General skills in Data Science, good software development skills, and a willingness to quickly develop new technical skills as required for the project

EECS 281(or equivalent) is required

Likely Majors: DATA, CS


General Coding
Specific Skills: General Programming skills, good software engineering practice and design, and a willingness to quickly develop new technical skills as required for the project 

EECS 281 (or equivalent) is required

Likely Majors: CS, DATA, BBA/CS



Additional Desired Skills/Knowledge/Experience
Successful team-based project experience.  Excellent interpersonal skills
Project Management utilizing Agile/Scrum
Experience in business process analysis
Interest in and general knowledge of Commercial Banking
Practical experience implementing predictive analytics in a complex data environment
Ability and desire to independently learn new technology skills as necessary for the project
Experience implementing large language models, neural networks and self-supervised / semi-supervised learning models
"""

In [25]:
from langchain.prompts import ChatPromptTemplate

In [28]:
prompt = ChatPromptTemplate.from_template("""
Given the below job description and the candidates relevant skills and experiences, provide a YES or NO answer about whether the candidate is a truly good fit for the job.
Job Description: {job_description}
Relevant Skills and Experiences: {context}
Answer:                                           
""")


# prompt = eval(prompt)
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

In [30]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


rag_chain = (
    {"job_description": RunnablePassthrough(), "context": retriever | format_docs}
    | prompt
    | llm
    | StrOutputParser()
)

In [31]:
rag_chain.invoke(job_description)

'YES'

In [ ]:
prompt = ChatPromptTemplate.from_template("""
Given the below job description and the candidates relevant skills and experiences, provide a YES or NO answer about whether the candidate is a truly good fit for the job.
Job Description: {job_description}
Relevant Skills and Experiences: {context}
Answer:                                           
""")

In [ ]:
rag_chain = (
    {"job_description": RunnablePassthrough(), "context": retriever | format_docs}
    | prompt
    | llm
    | StrOutputParser()
)